In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score


In [3]:
import math

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [5]:
tracking1 = pd.read_csv('./data/week1.csv')
tracking2 = pd.read_csv('./data/week2.csv')
tracking3 = pd.read_csv('./data/week3.csv')
tracking4 = pd.read_csv('./data/week4.csv')
tracking5 = pd.read_csv('./data/week5.csv')
tracking6 = pd.read_csv('./data/week6.csv')
tracking7 = pd.read_csv('./data/week7.csv')
tracking8 = pd.read_csv('./data/week8.csv')
tracking9 = pd.read_csv('./data/week9.csv')
tracking10 = pd.read_csv('./data/week10.csv')
tracking11 = pd.read_csv('./data/week11.csv')
tracking12 = pd.read_csv('./data/week12.csv')
tracking13 = pd.read_csv('./data/week13.csv')
tracking14 = pd.read_csv('./data/week14.csv')
tracking15 = pd.read_csv('./data/week15.csv')
tracking16 = pd.read_csv('./data/week16.csv')
tracking17 = pd.read_csv('./data/week17.csv')
tracking = pd.concat([tracking1, tracking2, tracking3, tracking4, tracking5
                      ,tracking6, tracking7, tracking8, tracking9, tracking10
                     ,tracking11, tracking12, tracking13, tracking14, tracking15
                     ,tracking16, tracking17], axis=0)

players = pd.read_csv('./data/players.csv')
plays = pd.read_csv('./data/plays.csv')

In [6]:
games = pd.read_csv('./data/games.csv')

In [7]:
def convert_to_inches(height):
    if isinstance(height, str) and '-' in height: 
        feet, inches = height.split('-')
        total_inches = int(feet) * 12 + int(inches)
        return total_inches
    else:
        return height

In [8]:
players['height'] = players['height'].apply(convert_to_inches)

In [9]:
players['height'].value_counts()

73    159
72    153
74    141
75    136
71    127
     ... 
78      2
79      2
66      1
68      1
67      1
Name: height, Length: 29, dtype: int64

# Convert game clock to seconds (not in use)

In [ ]:
#plays['gameClock'] = plays['gameClock'].fillna(plays['playDescription'].str.extract(r'\(?(\d{0,2}):(\d{2})\)?', expand=False).apply(lambda x: f"{x[0] if x[0] else '00'}:{x[1]}:00", axis=1))
#plays['gameClock']

In [ ]:
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

In [ ]:
#plays['gameClock_sec'] = plays['gameClock'].apply(strtoseconds)
#plays['gameClock_sec']

# Feature Engineering

## Get Score Differential (Home - Away)

In [10]:
plays = plays.merge(games, on=['gameId'])
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,offenseFormation,personnelO,defendersInTheBox,numberOfPassRushers,personnelD,typeDropback,preSnapVisitorScore,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,week
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,I_FORM,"2 RB, 1 TE, 2 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False,09/06/2018,20:20:00,PHI,ATL,1
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:10:00,49.0,NaN,NaN,I,0,0,-0.372360,False,09/06/2018,20:20:00,PHI,ATL,1
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,SHOTGUN,"2 RB, 1 TE, 2 WR",6.0,4.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0.0,0.0,13:05:00,49.0,NaN,NaN,I,0,0,-0.702779,False,09/06/2018,20:20:00,PHI,ATL,1
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE_ROLLOUT_LEFT,0.0,0.0,13:01:00,49.0,NaN,NaN,C,33,33,3.047530,False,09/06/2018,20:20:00,PHI,ATL,1
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,SHOTGUN,"2 RB, 3 TE, 0 WR",8.0,6.0,"6 DL, 3 LB, 2 DB",TRADITIONAL,0.0,0.0,10:59:00,11.0,NaN,NaN,I,0,0,-0.842272,False,09/06/2018,20:20:00,PHI,ATL,1


In [11]:
#impute missing preSnapHomeScores and preSnapvisitor Scores by taking average
#by quarter and by down
plays['preSnapHomeScore_imputed'] = plays.groupby(['homeTeamAbbr', 'quarter', 'down'])['preSnapHomeScore'].transform(lambda x: x.fillna(x.mean()))
plays['preSnapVisitorScore_imputed'] = plays.groupby(['visitorTeamAbbr', 'quarter', 'down'])['preSnapVisitorScore'].transform(lambda x: x.fillna(x.mean()))
plays['preSnapHomeScore_imputed']

0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
           ...    
19234    10.000000
19235    10.000000
19236     1.300000
19237     8.619048
19238    19.119048
Name: preSnapHomeScore_imputed, Length: 19239, dtype: float64

In [12]:
plays['score_diff_home_vs_away'] = (plays['preSnapHomeScore_imputed'] - plays['preSnapVisitorScore_imputed']).apply(np.ceil)
plays['score_diff_home_vs_away']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
19234   -12.0
19235   -12.0
19236    -1.0
19237     1.0
19238     9.0
Name: score_diff_home_vs_away, Length: 19239, dtype: float64

In [13]:
plays.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'playType', 'yardlineSide', 'yardlineNumber',
       'offenseFormation', 'personnelO', 'defendersInTheBox',
       'numberOfPassRushers', 'personnelD', 'typeDropback',
       'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
       'absoluteYardlineNumber', 'penaltyCodes', 'penaltyJerseyNumbers',
       'passResult', 'offensePlayResult', 'playResult', 'epa', 'isDefensivePI',
       'gameDate', 'gameTimeEastern', 'homeTeamAbbr', 'visitorTeamAbbr',
       'week', 'preSnapHomeScore_imputed', 'preSnapVisitorScore_imputed',
       'score_diff_home_vs_away'],
      dtype='object')

In [14]:
plays.drop(columns={'playDescription', 'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
                   'penaltyJerseyNumbers', 'isDefensivePI', 'gameDate', 'gameTimeEastern', 'week'}, inplace=True)

In [15]:
plays.shape

(19239, 26)

In [16]:
plays_no_nan = plays[plays['personnelO'].notna()]
plays_no_nan.shape

(19210, 26)

In [17]:
plays_no_nan = plays_no_nan[plays_no_nan['defendersInTheBox'].notna()]
plays_no_nan.shape

(19177, 26)

In [18]:
plays_no_nan['typeDropback'] = plays_no_nan['typeDropback'].fillna('UNKNOWN')
plays_no_nan['typeDropback'].value_counts()

TRADITIONAL               15645
SCRAMBLE_ROLLOUT_RIGHT     1096
SCRAMBLE                    677
UNKNOWN                     674
DESIGNED_ROLLOUT_RIGHT      482
SCRAMBLE_ROLLOUT_LEFT       462
DESIGNED_ROLLOUT_LEFT       141
Name: typeDropback, dtype: int64

In [19]:
plays_no_nan = plays_no_nan[plays_no_nan['offenseFormation'].notna()]
plays_no_nan.shape

(19098, 26)

In [20]:
plays_no_nan = plays_no_nan[plays_no_nan['numberOfPassRushers'].notna()]
plays_no_nan.shape

(18511, 26)

In [21]:
plays_no_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18511 entries, 0 to 19235
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gameId                       18511 non-null  int64  
 1   playId                       18511 non-null  int64  
 2   quarter                      18511 non-null  int64  
 3   down                         18511 non-null  int64  
 4   yardsToGo                    18511 non-null  int64  
 5   possessionTeam               18511 non-null  object 
 6   playType                     18511 non-null  object 
 7   yardlineSide                 18268 non-null  object 
 8   yardlineNumber               18511 non-null  int64  
 9   offenseFormation             18511 non-null  object 
 10  personnelO                   18511 non-null  object 
 11  defendersInTheBox            18511 non-null  float64
 12  numberOfPassRushers          18511 non-null  float64
 13  personnelD      

In [22]:
plays_no_nan.drop(columns={'yardlineSide', 'absoluteYardlineNumber'}, inplace=True)
plays_no_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18511 entries, 0 to 19235
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gameId                       18511 non-null  int64  
 1   playId                       18511 non-null  int64  
 2   quarter                      18511 non-null  int64  
 3   down                         18511 non-null  int64  
 4   yardsToGo                    18511 non-null  int64  
 5   possessionTeam               18511 non-null  object 
 6   playType                     18511 non-null  object 
 7   yardlineNumber               18511 non-null  int64  
 8   offenseFormation             18511 non-null  object 
 9   personnelO                   18511 non-null  object 
 10  defendersInTheBox            18511 non-null  float64
 11  numberOfPassRushers          18511 non-null  float64
 12  personnelD                   18511 non-null  object 
 13  typeDropback    

## Get Time to Throw (TTR)

In [23]:
def strtoseconds2(txt):
    txt = txt.split(':')
    sec, frac = txt[2].split('.')
    ans = int(txt[0])*3600 + int(txt[1])*60 + int(sec) + float('0.'+ frac)
    return ans

In [24]:
tracking['time'] = pd.to_datetime(tracking['time'])
tracking['time_only'] = tracking['time'].dt.strftime('%H:%M:%S.%f')
tracking['time_only']

0          01:07:14.599000
1          01:07:14.599000
2          01:07:14.599000
3          01:07:14.599000
4          01:07:14.599000
                ...       
1049260    04:23:24.200000
1049261    04:23:24.200000
1049262    04:23:24.200000
1049263    04:23:24.200000
1049264    04:23:24.200000
Name: time_only, Length: 18309388, dtype: object

In [25]:
tracking['time_sec'] = tracking['time_only'].apply(strtoseconds2)
tracking['time_sec']

0           4034.599
1           4034.599
2           4034.599
3           4034.599
4           4034.599
             ...    
1049260    15804.200
1049261    15804.200
1049262    15804.200
1049263    15804.200
1049264    15804.200
Name: time_sec, Length: 18309388, dtype: float64

In [26]:
snap_time = tracking[(tracking['event'] == 'ball_snap') & (tracking['position'] == 'QB')][['gameId', 'playId', 'time_sec']]
snap_time.rename(columns={'time_sec': 'snap_time_sec'}, inplace=True)

In [27]:
pass_time = tracking[(tracking['event'] == 'pass_forward') & (tracking['position'] == 'QB')][['gameId', 'playId','time_sec']]
pass_time.rename(columns={'time_sec': 'pass_time_sec'}, inplace=True)

In [28]:
throw_df = snap_time.merge(pass_time, on=['gameId', 'playId'])
throw_df.dropna(inplace=True)
throw_df['time_to_throw_sec'] = throw_df['pass_time_sec'] - throw_df['snap_time_sec']

In [29]:
throw_df.drop(columns={'snap_time_sec', 'pass_time_sec'})

,gameId,playId,time_to_throw_sec
0,2018090600,75,2.500
1,2018090600,146,2.000
2,2018090600,168,2.301
3,2018090600,190,2.801
4,2018090600,256,2.000
...,...,...,...
17943,2018123011,3947,3.200
17944,2018123011,4044,2.500
17945,2018123011,4089,3.200
17946,2018123011,4219,2.700


In [30]:
plays['playId'].value_counts()

51      45
73      15
3594    14
2994    14
3283    13
        ..
5196     1
5136     1
5095     1
4999     1
464      1
Name: playId, Length: 4592, dtype: int64

In [31]:
throw_df['playId'].value_counts()

51      45
98      15
1832    14
73      14
2715    14
        ..
1735     1
2059     1
2154     1
2628     1
4089     1
Name: playId, Length: 4537, dtype: int64

In [ ]:
del snap_time
del pass_time

## QB Dataframe at pass forward

In [32]:
qb_df = tracking[(tracking['event'] == 'pass_forward') & (tracking['position'] == 'QB')]

In [33]:
qb_df.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,time_only,time_sec
490,2018-09-07 01:07:18.099000+00:00,96.43,26.78,0.84,2.05,0.08,356.40,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,2018090600,75,left,NaN,01:07:18.099000,4038.099
1246,2018-09-07 01:09:12.700000+00:00,53.88,29.72,0.46,2.25,0.04,335.30,356.66,pass_forward,310.0,Matt Ryan,2.0,QB,31,away,2018090600,146,left,NaN,01:09:12.700000,4152.700
1974,2018-09-07 01:09:53.900000+00:00,55.11,28.71,1.65,0.48,0.17,250.68,154.41,pass_forward,310.0,Matt Ryan,2.0,QB,34,away,2018090600,168,left,NaN,01:09:53.900000,4193.900
2716,2018-09-07 01:10:38.400000+00:00,58.11,26.52,2.38,0.91,0.24,260.91,179.38,pass_forward,310.0,Matt Ryan,2.0,QB,39,away,2018090600,190,left,NaN,01:10:38.400000,4238.400
3566,2018-09-07 01:12:40.500000+00:00,16.97,24.06,0.90,0.50,0.09,356.00,355.19,pass_forward,310.0,Matt Ryan,2.0,QB,31,away,2018090600,256,left,NaN,01:12:40.500000,4360.500


## Get Targeted Receiver + Weight and Height Stats

In [34]:
tracking['event'].unique()

array(['None', 'ball_snap', 'pass_forward', 'pass_arrived',
       'pass_outcome_caught', 'out_of_bounds', 'pass_outcome_incomplete',
       'first_contact', 'tackle', 'man_in_motion', 'play_action',
       'qb_sack', 'fumble', 'fumble_offense_recovered', 'handoff',
       'pass_tipped', 'pass_outcome_interception', 'qb_strip_sack',
       'pass_shovel', 'line_set', 'shift', 'touchdown',
       'fumble_defense_recovered', 'pass_outcome_touchdown', 'run',
       'touchback', 'penalty_flag', 'penalty_accepted', 'qb_spike',
       'field_goal_blocked', 'punt_fake', 'snap_direct',
       'run_pass_option', 'pass_lateral', 'lateral', 'field_goal_fake',
       'huddle_start_offense', 'huddle_break_offense', 'timeout_home',
       'safety', 'field_goal_play'], dtype=object)

In [35]:
defense_positions = ['SS', 'FS', 'MLB', 'CB', 'LB', 'OLB', 'ILB', 'DL', 'DB', 'NT', 'S', 'DE']
pass_arrived_time = ['pass_arrived', 'pass_outcome_caught', 'pass_outcome_incomplete', 'pass_outcome_interception', 'pass_outcome_touchdown']

In [36]:
#get tracking records when the pass reaches it's endpoint
pass_arrived = tracking[tracking['event'].isin(['pass_arrived', 'pass_outcome_caught', 'pass_outcome_incomplete', 'pass_outcome_interception', 'pass_outcome_touchdown'])]
#get ball position
ball_positions = pass_arrived[pass_arrived['nflId'].isna()][['gameId', 'playId', 'x', 'y']].rename(columns={'x': 'ball_x', 'y': 'ball_y'})
pass_arrived = pass_arrived.merge(ball_positions, on=['gameId', 'playId'], how='left')
#get each player's position from the ball
pass_arrived['distance_to_ball'] = np.sqrt((pass_arrived['x'] - pass_arrived['ball_x'])**2 + (pass_arrived['y'] - pass_arrived['ball_y'])**2)


In [37]:
#get players not in defense positions
pass_arrived = pass_arrived[~pass_arrived['position'].isin(defense_positions)]
#remove nan rows
pass_arrived = pass_arrived[pass_arrived['nflId'].notna()]
pass_arrived = pass_arrived[pass_arrived['distance_to_ball'].notna()]

In [38]:
closest_players = pass_arrived.loc[pass_arrived.groupby(['gameId', 'playId'])['distance_to_ball'].idxmin()][['gameId', 'playId', 'nflId', 'distance_to_ball']]
closest_players = closest_players.rename(columns={'nflId':'target_receiver_nflId'})
closest_players.rename(columns={'distance_to_ball':'targeted_receiver_distance_to_ball'}, inplace=True)

In [39]:
closest_players.head()

,gameId,playId,target_receiver_nflId,targeted_receiver_distance_to_ball
33,2018090600,75,2495454.0,0.751066
80,2018090600,146,2560854.0,0.656049
117,2018090600,168,2543583.0,4.420882
159,2018090600,190,2495454.0,0.351283
215,2018090600,256,2543583.0,1.287633


In [40]:
closest_players = closest_players.merge(players[['nflId','height', 'weight']], left_on=['target_receiver_nflId'], right_on=['nflId'])


In [41]:
closest_players.rename(columns={'height': 'targeted_receiver_height', 'weight': 'targeted_receiver_weight'})

,gameId,playId,target_receiver_nflId,targeted_receiver_distance_to_ball,nflId,targeted_receiver_height,targeted_receiver_weight
0,2018090600,75,2495454.0,0.751066,2495454,75,220
1,2018090600,190,2495454.0,0.351283,2495454,75,220
2,2018090600,545,2495454.0,11.331867,2495454,75,220
3,2018090600,889,2495454.0,4.531247,2495454,75,220
4,2018090600,1423,2495454.0,0.196977,2495454,75,220
...,...,...,...,...,...,...,...
17830,2018123012,3412,2561359.0,0.511957,2561359,72,180
17831,2018123012,564,264.0,4.169904,264,75,219
17832,2018123013,1861,2559485.0,0.176918,2559485,72,198
17833,2018123013,4102,2559485.0,1.389568,2559485,72,198


In [42]:
tracking = tracking.merge(closest_players, on=['gameId', 'playId'], how='left')
tracking.head()

,time,x,y,s,a,dis,o,dir,event,nflId_x,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,time_only,time_sec,target_receiver_nflId,targeted_receiver_distance_to_ball,nflId_y,height,weight
0,2018-09-07 01:07:14.599000+00:00,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,2495454.0,75,220.0
1,2018-09-07 01:07:14.599000+00:00,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,2495454.0,75,220.0
2,2018-09-07 01:07:14.599000+00:00,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH,01:07:14.599000,4034.599,2495454.0,0.751066,2495454.0,75,220.0
3,2018-09-07 01:07:14.599000+00:00,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,2495454.0,75,220.0
4,2018-09-07 01:07:14.599000+00:00,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH,01:07:14.599000,4034.599,2495454.0,0.751066,2495454.0,75,220.0


In [43]:
tracking.drop(columns = {'nflId_y'}, inplace=True)
tracking.rename(columns = {'nflId_x': 'nflId'},inplace=True)

## Get Target Receiver Position When Ball Thrown + Route

In [44]:
pass_forward = tracking[tracking['event'] == 'pass_forward']
pass_forward.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,time_only,time_sec,target_receiver_nflId,targeted_receiver_distance_to_ball,height,weight
490,2018-09-07 01:07:18.099000+00:00,96.43,26.78,0.84,2.05,0.08,356.40,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,2018090600,75,left,NaN,01:07:18.099000,4038.099,2495454.0,0.751066,75,220.0
491,2018-09-07 01:07:18.099000+00:00,84.82,38.33,5.53,4.30,0.54,295.34,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,2018090600,75,left,NaN,01:07:18.099000,4038.099,2495454.0,0.751066,75,220.0
492,2018-09-07 01:07:18.099000+00:00,80.69,44.91,3.11,6.67,0.34,155.11,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,2018090600,75,left,HITCH,01:07:18.099000,4038.099,2495454.0,0.751066,75,220.0
493,2018-09-07 01:07:18.099000+00:00,86.02,18.64,3.52,0.69,0.35,28.19,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,2018090600,75,left,NaN,01:07:18.099000,4038.099,2495454.0,0.751066,75,220.0
494,2018-09-07 01:07:18.099000+00:00,82.65,34.56,6.35,1.21,0.64,262.06,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,2018090600,75,left,HITCH,01:07:18.099000,4038.099,2495454.0,0.751066,75,220.0


In [45]:
targeted_receiver_positions = pass_forward[pass_forward['target_receiver_nflId'] == pass_forward['nflId']][['gameId', 'playId', 'target_receiver_nflId','x', 'y', 's','a', 'dis', 'o', 'dir', 'route']]
targeted_receiver_positions.head()


,gameId,playId,target_receiver_nflId,x,y,s,a,dis,o,dir,route
492,2018090600,75,2495454.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
1258,2018090600,146,2560854.0,45.51,36.91,2.61,2.25,0.27,132.42,254.43,HITCH
1979,2018090600,168,2543583.0,51.83,13.69,4.58,2.44,0.46,316.70,197.08,SCREEN
2718,2018090600,190,2495454.0,35.70,21.07,7.32,4.19,0.73,215.00,252.39,CORNER
3570,2018090600,256,2543583.0,9.17,35.20,5.11,3.28,0.50,62.66,349.05,OUT


In [46]:
targeted_receiver_positions = targeted_receiver_positions.rename(columns={'x':'targeted_receiver_pass_forward_x', 
                                            'y':'targeted_receiver_pass_forward_y', 
                                            's':'targeted_receiver_pass_forward_s', 
                                            'a':'targeted_receiver_pass_forward_a', 
                                            'dis':'targeted_receiver_pass_forward_dis', 
                                            'o':'targeted_receiver_pass_forward_o', 
                                            'dir':'targeted_receiver_pass_forward_dir',
                                            'route': 'targeted_receiver_route'})
targeted_receiver_positions.head()

,gameId,playId,target_receiver_nflId,targeted_receiver_pass_forward_x,targeted_receiver_pass_forward_y,targeted_receiver_pass_forward_s,targeted_receiver_pass_forward_a,targeted_receiver_pass_forward_dis,targeted_receiver_pass_forward_o,targeted_receiver_pass_forward_dir,targeted_receiver_route
492,2018090600,75,2495454.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
1258,2018090600,146,2560854.0,45.51,36.91,2.61,2.25,0.27,132.42,254.43,HITCH
1979,2018090600,168,2543583.0,51.83,13.69,4.58,2.44,0.46,316.70,197.08,SCREEN
2718,2018090600,190,2495454.0,35.70,21.07,7.32,4.19,0.73,215.00,252.39,CORNER
3570,2018090600,256,2543583.0,9.17,35.20,5.11,3.28,0.50,62.66,349.05,OUT


In [47]:
tracking.shape

(18309388, 25)

In [48]:
tracking = tracking.merge(targeted_receiver_positions, on=['gameId', 'playId', 'target_receiver_nflId'])
tracking.shape

(17012731, 33)

In [49]:
tracking.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,time_only,time_sec,target_receiver_nflId,targeted_receiver_distance_to_ball,height,weight,targeted_receiver_pass_forward_x,targeted_receiver_pass_forward_y,targeted_receiver_pass_forward_s,targeted_receiver_pass_forward_a,targeted_receiver_pass_forward_dis,targeted_receiver_pass_forward_o,targeted_receiver_pass_forward_dir,targeted_receiver_route
0,2018-09-07 01:07:14.599000+00:00,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
1,2018-09-07 01:07:14.599000+00:00,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
2,2018-09-07 01:07:14.599000+00:00,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
3,2018-09-07 01:07:14.599000+00:00,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
4,2018-09-07 01:07:14.599000+00:00,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH


## Get Top 3 Closest Defenders to Targeted Receiver at Pass Arrived

In [50]:
tracking['position'].unique()

array(['QB', 'SS', 'WR', 'FS', 'RB', 'MLB', 'CB', 'TE', 'LB', 'FB', nan,
       'OLB', 'HB', 'ILB', 'DL', 'DB', 'NT', 'S', 'DE', 'P', 'LS', 'K',
       'DT'], dtype=object)

In [51]:
defense_positions_non_line = ['SS', 'FS', 'MLB', 'CB', 'LB', 'OLB', 'ILB', 'DB', 'S']

In [52]:
offensive_positions = ['WR', 'RB', 'TE', 'FB','HB']

In [53]:
pass_arrived = tracking[tracking['event'].isin(['pass_arrived'])]

pass_arrived = pass_arrived[pass_arrived['position'].isin(defense_positions_non_line)]
pass_arrived = pass_arrived[pass_arrived['nflId'].notna()]
pass_arrived['defender_distance_from_target_receiver'] = np.sqrt((pass_arrived['x'] - pass_arrived['targeted_receiver_pass_forward_x'])**2 + (pass_arrived['y'] - pass_arrived['targeted_receiver_pass_forward_y'])**2)
#closest_players = pass_forward.loc[pass_forward.groupby(['gameId', 'playId'])['closest_defender_distance_from_target_receiver'].idxmin()][['gameId', 'playId', 'nflId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'closest_defender_distance_from_target_receiver']]
pass_arrived.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,time_only,time_sec,target_receiver_nflId,targeted_receiver_distance_to_ball,height,weight,targeted_receiver_pass_forward_x,targeted_receiver_pass_forward_y,targeted_receiver_pass_forward_s,targeted_receiver_pass_forward_a,targeted_receiver_pass_forward_dis,targeted_receiver_pass_forward_o,targeted_receiver_pass_forward_dir,targeted_receiver_route,defender_distance_from_target_receiver
645,2018-09-07 01:07:19.200000+00:00,82.34,45.18,7.57,1.72,0.75,354.68,353.52,pass_arrived,79848.0,Malcolm Jenkins,27.0,SS,47,home,2018090600,75,left,NaN,01:07:19.200000,4039.2,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH,1.671945
647,2018-09-07 01:07:19.200000+00:00,82.90,17.62,3.64,4.01,0.36,13.28,299.37,pass_arrived,2495613.0,Corey Graham,24.0,FS,47,home,2018090600,75,left,NaN,01:07:19.200000,4039.2,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH,27.379339
649,2018-09-07 01:07:19.200000+00:00,70.28,35.09,6.10,2.74,0.59,32.94,29.19,pass_arrived,2534832.0,Rodney McLeod,23.0,FS,47,home,2018090600,75,left,NaN,01:07:19.200000,4039.2,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH,14.310853
651,2018-09-07 01:07:19.200000+00:00,82.76,36.69,4.85,2.33,0.48,15.09,326.23,pass_arrived,2552315.0,Jordan Hicks,58.0,MLB,47,home,2018090600,75,left,NaN,01:07:19.200000,4039.2,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH,8.476633
652,2018-09-07 01:07:19.200000+00:00,77.11,17.30,4.83,3.10,0.47,167.90,193.72,pass_arrived,2552689.0,Ronald Darby,21.0,CB,47,home,2018090600,75,left,NaN,01:07:19.200000,4039.2,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH,27.841130


In [54]:
pass_arrived['rank'] = pass_arrived.groupby(['gameId', 'playId'])['defender_distance_from_target_receiver'].rank(ascending=True, method='dense').astype(int)
closest_3_defenders = pass_arrived[pass_arrived['rank'] <= 3]
closest_3_defenders = closest_3_defenders[['gameId', 'playId', 'nflId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'defender_distance_from_target_receiver', 'rank']]
closest_3_defenders.head()

,gameId,playId,nflId,x,y,s,a,dis,o,dir,defender_distance_from_target_receiver,rank
645,2018090600,75,79848.0,82.34,45.18,7.57,1.72,0.75,354.68,353.52,1.671945,1
651,2018090600,75,2552315.0,82.76,36.69,4.85,2.33,0.48,15.09,326.23,8.476633,3
653,2018090600,75,2555383.0,74.40,47.11,4.05,4.64,0.38,69.31,13.45,6.663640,2
1317,2018090600,146,79848.0,45.08,43.86,6.25,1.03,0.62,30.67,335.31,6.963289,3
1320,2018090600,146,2552315.0,44.24,34.98,4.96,2.41,0.49,65.47,6.97,2.310368,1


In [55]:
closest_3_defenders = closest_3_defenders.merge(players[['nflId','height', 'weight']], on=['nflId'])

In [56]:
closest_3_defenders_pivoted = closest_3_defenders.pivot_table(index=['gameId', 'playId'],
                                                              columns = 'rank',
                                                              values = [ 'weight', 'height','x', 'y', 's', 'a', 'dis', 'o', 'dir', 'defender_distance_from_target_receiver'],
                                                              aggfunc = 'first'
                                                             )
closest_3_defenders_pivoted.columns = [f'{col[0]}_{col[1]}_def_from_target_receiver' for col in closest_3_defenders_pivoted.columns]

# Reset the index to make it a proper DataFrame again
closest_3_defenders_pivoted.reset_index(inplace=True)

closest_3_defenders_pivoted.head()

,gameId,playId,a_1_def_from_target_receiver,a_2_def_from_target_receiver,a_3_def_from_target_receiver,defender_distance_from_target_receiver_1_def_from_target_receiver,defender_distance_from_target_receiver_2_def_from_target_receiver,defender_distance_from_target_receiver_3_def_from_target_receiver,dir_1_def_from_target_receiver,dir_2_def_from_target_receiver,dir_3_def_from_target_receiver,dis_1_def_from_target_receiver,dis_2_def_from_target_receiver,dis_3_def_from_target_receiver,height_1_def_from_target_receiver,height_2_def_from_target_receiver,height_3_def_from_target_receiver,o_1_def_from_target_receiver,o_2_def_from_target_receiver,o_3_def_from_target_receiver,s_1_def_from_target_receiver,s_2_def_from_target_receiver,s_3_def_from_target_receiver,weight_1_def_from_target_receiver,weight_2_def_from_target_receiver,weight_3_def_from_target_receiver,x_1_def_from_target_receiver,x_2_def_from_target_receiver,x_3_def_from_target_receiver,y_1_def_from_target_receiver,y_2_def_from_target_receiver,y_3_def_from_target_receiver
0,2018090600,75,1.72,4.64,2.33,1.671945,6.663640,8.476633,353.52,13.45,326.23,0.75,0.38,0.48,72,72,73,354.68,69.31,15.09,7.57,4.05,4.85,204,191,236,82.34,74.40,82.76,45.18,47.11,36.69
1,2018090600,146,2.41,1.44,1.03,2.310368,2.446753,6.963289,6.97,111.07,335.31,0.49,0.48,0.62,73,72,72,65.47,133.50,30.67,4.96,4.98,6.25,236,191,204,44.24,43.30,45.08,34.98,37.96,43.86
2,2018090600,190,2.39,0.82,1.53,4.520896,7.394714,7.503666,271.12,234.96,228.66,0.31,0.58,0.27,72,72,70,238.67,250.59,235.39,3.11,5.82,2.70,204,181,195,36.78,33.23,43.17,25.46,14.10,20.36
3,2018090600,256,1.33,1.19,1.25,2.102380,3.107105,9.129562,5.11,5.28,0.64,0.50,0.72,0.54,73,74,74,1.63,4.85,4.57,4.89,7.27,5.33,236,230,230,7.83,8.96,8.22,33.58,38.30,26.12
4,2018090600,320,2.39,2.76,3.18,3.848870,5.362406,6.921691,1.61,329.15,4.91,0.52,0.70,0.49,73,70,75,319.32,286.86,347.37,5.31,7.09,4.99,216,186,232,19.87,18.99,20.51,39.93,46.63,36.19


# Get 5 Closest Defenders from QB at Pass Throw

In [57]:
qb_df_position = qb_df[['gameId', 'playId','x', 'y',  's', 'a', 'dis', 'o', 'dir']]
qb_df_position.rename(columns={'x': 'qb_x', 'y': 'qb_y', 's': 'qb_s', 
                               'a': 'qb_a', 'dis':'qb_dis', 'o': 'qb_o', 'dir': 'qb_dir'}, inplace=True)
qb_df_position.head()

C:\Users\Doug\AppData\Local\Temp\ipykernel_11936\3746376049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_df_position.rename(columns={'x': 'qb_x', 'y': 'qb_y', 's': 'qb_s',


,gameId,playId,qb_x,qb_y,qb_s,qb_a,qb_dis,qb_o,qb_dir
490,2018090600,75,96.43,26.78,0.84,2.05,0.08,356.40,333.39
1246,2018090600,146,53.88,29.72,0.46,2.25,0.04,335.30,356.66
1974,2018090600,168,55.11,28.71,1.65,0.48,0.17,250.68,154.41
2716,2018090600,190,58.11,26.52,2.38,0.91,0.24,260.91,179.38
3566,2018090600,256,16.97,24.06,0.90,0.50,0.09,356.00,355.19


In [58]:
pass_forward_def = tracking[tracking['event']=='pass_forward']
pass_forward_def = pass_forward_def[pass_forward_def['position'].isin(defense_positions_non_line)]
pass_forward_def = pass_forward_def[['gameId', 'playId','x', 'y',  's', 'a', 'dis', 'o', 'dir', 'nflId']]

In [59]:
pass_forward_def.head()

,gameId,playId,x,y,s,a,dis,o,dir,nflId
491,2018090600,75,84.82,38.33,5.53,4.30,0.54,295.34,318.58,79848.0
493,2018090600,75,86.02,18.64,3.52,0.69,0.35,28.19,216.15,2495613.0
495,2018090600,75,69.63,30.81,2.93,2.65,0.30,79.24,315.98,2534832.0
497,2018090600,75,86.10,33.53,3.75,1.49,0.37,52.23,311.29,2552315.0
498,2018090600,75,79.51,20.00,2.99,0.69,0.30,78.25,259.75,2552689.0


In [60]:
qb_def_df =  pass_forward_def.merge(qb_df_position, on=['gameId', 'playId'])
qb_def_df['distance_from_qb'] = np.sqrt((qb_def_df['x'] - qb_def_df['qb_x'])**2 + (qb_def_df['y'] - qb_def_df['qb_y'])**2)
qb_def_df.head()

,gameId,playId,x,y,s,a,dis,o,dir,nflId,qb_x,qb_y,qb_s,qb_a,qb_dis,qb_o,qb_dir,distance_from_qb
0,2018090600,75,84.82,38.33,5.53,4.30,0.54,295.34,318.58,79848.0,96.43,26.78,0.84,2.05,0.08,356.4,333.39,16.376648
1,2018090600,75,86.02,18.64,3.52,0.69,0.35,28.19,216.15,2495613.0,96.43,26.78,0.84,2.05,0.08,356.4,333.39,13.214677
2,2018090600,75,69.63,30.81,2.93,2.65,0.30,79.24,315.98,2534832.0,96.43,26.78,0.84,2.05,0.08,356.4,333.39,27.101308
3,2018090600,75,86.10,33.53,3.75,1.49,0.37,52.23,311.29,2552315.0,96.43,26.78,0.84,2.05,0.08,356.4,333.39,12.339830
4,2018090600,75,79.51,20.00,2.99,0.69,0.30,78.25,259.75,2552689.0,96.43,26.78,0.84,2.05,0.08,356.4,333.39,18.227858


In [61]:
qb_def_df['rank'] = qb_def_df.groupby(['gameId', 'playId'])['distance_from_qb'].rank(ascending=True, method='dense').astype(int)


In [62]:
closest_5_def_qb = qb_def_df[qb_def_df['rank'] <= 5]
closest_5_def_qb = closest_5_def_qb[['gameId', 'playId', 'nflId', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_from_qb', 'rank']]
closest_5_def_qb.head()

,gameId,playId,nflId,x,y,s,a,dis,o,dir,distance_from_qb,rank
0,2018090600,75,79848.0,84.82,38.33,5.53,4.30,0.54,295.34,318.58,16.376648,4
1,2018090600,75,2495613.0,86.02,18.64,3.52,0.69,0.35,28.19,216.15,13.214677,2
3,2018090600,75,2552315.0,86.10,33.53,3.75,1.49,0.37,52.23,311.29,12.339830,1
4,2018090600,75,2552689.0,79.51,20.00,2.99,0.69,0.30,78.25,259.75,18.227858,5
6,2018090600,75,2558175.0,83.06,25.98,4.08,3.73,0.40,42.57,303.26,13.393913,3


In [63]:
closest_5_def_qb = closest_5_def_qb.merge(players[['nflId','height', 'weight']], on=['nflId'])

In [64]:
closest_5_def_qb.head()

,gameId,playId,nflId,x,y,s,a,dis,o,dir,distance_from_qb,rank,height,weight
0,2018090600,75,79848.0,84.82,38.33,5.53,4.30,0.54,295.34,318.58,16.376648,4,72,204
1,2018090600,146,79848.0,46.18,41.10,5.36,2.25,0.53,63.48,343.51,13.740247,4,72,204
2,2018090600,168,79848.0,42.68,23.14,3.48,2.20,0.35,57.96,298.31,13.620932,3,72,204
3,2018090600,190,79848.0,39.77,22.27,1.14,4.50,0.10,83.78,338.71,18.825995,3,72,204
4,2018090600,256,79848.0,7.91,47.81,3.18,2.24,0.34,84.55,316.81,25.419404,5,72,204


In [65]:
closest_5_def_qb_pivoted = closest_5_def_qb.pivot_table(index=['gameId', 'playId'],
                                                              columns = 'rank',
                                                              values = [ 'height', 'weight','x', 'y', 's', 'a', 'dis', 'o', 'dir', 'distance_from_qb'],
                                                              aggfunc = 'first'
                                                             )
closest_5_def_qb_pivoted.columns = [f'{col[0]}_{col[1]}_def_from_qb' for col in closest_5_def_qb_pivoted.columns]

# Reset the index to make it a proper DataFrame again
closest_5_def_qb_pivoted.reset_index(inplace=True)

closest_5_def_qb_pivoted.head()

,gameId,playId,a_1_def_from_qb,a_2_def_from_qb,a_3_def_from_qb,a_4_def_from_qb,a_5_def_from_qb,dir_1_def_from_qb,dir_2_def_from_qb,dir_3_def_from_qb,dir_4_def_from_qb,dir_5_def_from_qb,dis_1_def_from_qb,dis_2_def_from_qb,dis_3_def_from_qb,dis_4_def_from_qb,dis_5_def_from_qb,distance_from_qb_1_def_from_qb,distance_from_qb_2_def_from_qb,distance_from_qb_3_def_from_qb,distance_from_qb_4_def_from_qb,distance_from_qb_5_def_from_qb,height_1_def_from_qb,height_2_def_from_qb,height_3_def_from_qb,height_4_def_from_qb,height_5_def_from_qb,o_1_def_from_qb,o_2_def_from_qb,o_3_def_from_qb,o_4_def_from_qb,o_5_def_from_qb,s_1_def_from_qb,s_2_def_from_qb,s_3_def_from_qb,s_4_def_from_qb,s_5_def_from_qb,weight_1_def_from_qb,weight_2_def_from_qb,weight_3_def_from_qb,weight_4_def_from_qb,weight_5_def_from_qb,x_1_def_from_qb,x_2_def_from_qb,x_3_def_from_qb,x_4_def_from_qb,x_5_def_from_qb,y_1_def_from_qb,y_2_def_from_qb,y_3_def_from_qb,y_4_def_from_qb,y_5_def_from_qb
0,2018090600,75,1.49,0.69,3.73,4.30,0.69,311.29,216.15,303.26,318.58,259.75,0.37,0.35,0.40,0.54,0.30,12.339830,13.214677,13.393913,16.376648,18.227858,73,72,74,72,71,52.23,28.19,42.57,295.34,78.25,3.75,3.52,4.08,5.53,2.99,236.0,196.0,230.0,204.0,193.0,86.10,86.02,83.06,84.82,79.51,33.53,18.64,25.98,38.33,20.00
1,2018090600,146,3.32,2.61,3.24,2.25,4.67,326.78,349.85,286.78,343.51,124.72,0.16,0.35,0.31,0.53,0.33,8.918100,10.095435,10.544126,13.740247,15.607181,72,73,74,72,72,67.18,74.39,56.57,63.48,113.51,1.45,3.62,3.15,5.36,3.52,181.0,236.0,230.0,204.0,191.0,46.14,44.25,43.83,46.18,41.28,25.29,32.75,26.53,41.10,38.93
2,2018090600,168,6.34,1.71,2.20,2.80,2.41,235.39,13.14,298.31,170.83,237.23,0.12,0.20,0.35,0.41,0.13,8.280121,9.913314,13.620932,18.162379,20.934997,73,74,72,72,72,121.10,157.73,57.96,93.06,109.56,1.59,1.95,3.48,4.14,1.41,236.0,230.0,204.0,181.0,191.0,47.21,47.74,42.68,46.59,39.82,31.19,35.34,23.14,12.67,43.01
3,2018090600,190,1.30,2.47,4.50,0.71,1.23,94.63,157.72,338.71,255.31,268.22,0.34,0.51,0.10,0.32,0.33,3.420658,15.306329,18.825995,20.089121,21.892960,73,70,72,72,72,4.87,112.27,83.78,79.98,177.96,3.32,4.99,1.14,3.21,3.30,236.0,195.0,204.0,181.0,191.0,55.56,42.85,39.77,40.19,40.27,24.24,25.33,22.27,17.44,39.21
4,2018090600,256,3.37,0.82,3.15,1.87,2.24,358.06,10.29,15.30,215.17,316.81,0.41,0.49,0.30,0.55,0.34,8.934607,10.938976,12.746584,19.940203,25.419404,74,73,74,70,72,29.16,35.35,47.22,149.55,84.55,4.23,4.98,2.65,5.62,3.18,230.0,236.0,230.0,195.0,204.0,8.41,7.21,8.20,7.43,7.91,21.50,29.00,33.31,6.55,47.81


## Get Pass Distance from QB to targeted Receiver

In [66]:
tracking.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route,time_only,time_sec,target_receiver_nflId,targeted_receiver_distance_to_ball,height,weight,targeted_receiver_pass_forward_x,targeted_receiver_pass_forward_y,targeted_receiver_pass_forward_s,targeted_receiver_pass_forward_a,targeted_receiver_pass_forward_dis,targeted_receiver_pass_forward_o,targeted_receiver_pass_forward_dir,targeted_receiver_route
0,2018-09-07 01:07:14.599000+00:00,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
1,2018-09-07 01:07:14.599000+00:00,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
2,2018-09-07 01:07:14.599000+00:00,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
3,2018-09-07 01:07:14.599000+00:00,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH
4,2018-09-07 01:07:14.599000+00:00,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH,01:07:14.599000,4034.599,2495454.0,0.751066,75,220.0,80.69,44.91,3.11,6.67,0.34,155.11,297.88,HITCH


In [67]:
pass_arrived_time = ['pass_arrived']

In [68]:
football_pass_forward = tracking[(tracking['position'] == 'QB') & (tracking['event'] == 'pass_forward')][['gameId', 'playId', 'x', 'y']]
receiver_pass_arrival= tracking[(tracking['nflId'] == tracking['target_receiver_nflId']) & (tracking['event'].isin(pass_arrived_time))][['gameId', 'playId', 'x', 'y']]
football_pass_forward.head()

,gameId,playId,x,y
490,2018090600,75,96.43,26.78
1246,2018090600,146,53.88,29.72
1974,2018090600,168,55.11,28.71
2716,2018090600,190,58.11,26.52
3566,2018090600,256,16.97,24.06


In [69]:
receiver_pass_arrival.head()

,gameId,playId,x,y
646,2018090600,75,81.11,47.87
1328,2018090600,146,44.79,36.79
2956,2018090600,190,25.99,12.13
3678,2018090600,256,8.87,40.65
4318,2018090600,320,17.58,46.78


In [70]:
throw_distance_df = receiver_pass_arrival.merge(football_pass_forward, on=['gameId', 'playId'])
throw_distance_df.head()

,gameId,playId,x_x,y_x,x_y,y_y
0,2018090600,75,81.11,47.87,96.43,26.78
1,2018090600,146,44.79,36.79,53.88,29.72
2,2018090600,190,25.99,12.13,58.11,26.52
3,2018090600,256,8.87,40.65,16.97,24.06
4,2018090600,320,17.58,46.78,7.79,30.68


In [71]:
throw_distance_df['throw_distance'] = np.sqrt((throw_distance_df['x_x'] - throw_distance_df['x_y'])**2 + (throw_distance_df['y_x'] - throw_distance_df['y_y'])**2)
throw_distance_df.head()

,gameId,playId,x_x,y_x,x_y,y_y,throw_distance
0,2018090600,75,81.11,47.87,96.43,26.78,26.067039
1,2018090600,146,44.79,36.79,53.88,29.72,11.515772
2,2018090600,190,25.99,12.13,58.11,26.52,35.196115
3,2018090600,256,8.87,40.65,16.97,24.06,18.461801
4,2018090600,320,17.58,46.78,7.79,30.68,18.842879


# Merge all Data Frames to Play Data Frame

In [ ]:
del data

In [72]:
data = plays_no_nan.copy()

In [73]:
data.columns

Index(['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'playType', 'yardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInTheBox', 'numberOfPassRushers', 'personnelD',
       'typeDropback', 'penaltyCodes', 'passResult', 'offensePlayResult',
       'playResult', 'epa', 'homeTeamAbbr', 'visitorTeamAbbr',
       'preSnapHomeScore_imputed', 'preSnapVisitorScore_imputed',
       'score_diff_home_vs_away'],
      dtype='object')

In [74]:
data.shape

(18511, 24)

In [75]:
data = data[['gameId', 'playId', 'quarter', 'down', 
             'yardsToGo', 'offenseFormation', 'personnelO','defendersInTheBox',
            'numberOfPassRushers', 'personnelD', 'typeDropback','passResult',
            'score_diff_home_vs_away']]
data.shape

(18511, 13)

In [76]:
data = data.merge(closest_5_def_qb_pivoted, on=['gameId', 'playId'])
data = data.merge(closest_3_defenders_pivoted, on=['gameId', 'playId'])
data.shape

(14134, 93)

In [77]:
data = data.merge(throw_df[['gameId', 'playId', 'time_to_throw_sec']], on=['gameId', 'playId'])
data.shape

(14329, 94)

In [78]:
data.drop_duplicates(inplace=True)
data.shape

(14137, 94)

In [79]:
data = data.merge(throw_distance_df, on=['gameId', 'playId'])
data.shape

(14262, 99)

In [80]:
targeted_receiver_positions.shape

(17730, 11)

In [81]:
data = data.merge(targeted_receiver_positions, on=['gameId', 'playId'])
data.shape

(14326, 108)

In [82]:
data = data.merge(closest_players[['gameId', 'playId', 'targeted_receiver_distance_to_ball']], on=['gameId', 'playId'])
data.shape

(14326, 109)

In [83]:
data.columns

Index(['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'offenseFormation',
       'personnelO', 'defendersInTheBox', 'numberOfPassRushers', 'personnelD',
       ...
       'target_receiver_nflId', 'targeted_receiver_pass_forward_x',
       'targeted_receiver_pass_forward_y', 'targeted_receiver_pass_forward_s',
       'targeted_receiver_pass_forward_a',
       'targeted_receiver_pass_forward_dis',
       'targeted_receiver_pass_forward_o',
       'targeted_receiver_pass_forward_dir', 'targeted_receiver_route',
       'targeted_receiver_distance_to_ball'],
      dtype='object', length=109)

In [84]:
data['numberOfPassRushers'] = data['numberOfPassRushers'].astype(str) + ' pass rushers'
data['defendersInTheBox'] = data['defendersInTheBox'].astype(str) + ' defenders'
data['quarter'] = 'Quarter ' + data['quarter'].astype(str)
data['down'] = 'Down ' + data['down'].astype(str)

In [85]:
data.columns

Index(['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'offenseFormation',
       'personnelO', 'defendersInTheBox', 'numberOfPassRushers', 'personnelD',
       ...
       'target_receiver_nflId', 'targeted_receiver_pass_forward_x',
       'targeted_receiver_pass_forward_y', 'targeted_receiver_pass_forward_s',
       'targeted_receiver_pass_forward_a',
       'targeted_receiver_pass_forward_dis',
       'targeted_receiver_pass_forward_o',
       'targeted_receiver_pass_forward_dir', 'targeted_receiver_route',
       'targeted_receiver_distance_to_ball'],
      dtype='object', length=109)

In [86]:
cat_cols = ['offenseFormation', 'personnelO', 'personnelD','typeDropback', 
            'targeted_receiver_route','down', 'quarter',
             'defendersInTheBox','numberOfPassRushers']
        
for col in cat_cols:
    data = pd.concat([data, pd.get_dummies(data[col])], axis = 1)
    data = data.drop(col,axis=1)

In [87]:
#data['closest_defender_distance_from_pass_forward_difference'] = data['targeted_receiver_distance_from_pass_forward'] - data['defender_distance_from_target_receiver_1_def_from_target_receiver']
data['targeted_receiver_pass_forward_o_dir_diff'] = data['targeted_receiver_pass_forward_o'] - data['targeted_receiver_pass_forward_dir']
data['closest_defender_pass_forward_o_dir_diff'] = data['o_1_def_from_target_receiver'] - data['dir_1_def_from_target_receiver']
data['targeted_closest_dir_difference'] = data['dir_1_def_from_target_receiver'] - data['targeted_receiver_pass_forward_dir']
data['targeted_closest_dir_difference'] = data['o_1_def_from_target_receiver'] - data['targeted_receiver_pass_forward_o']

In [88]:
data = data[data['passResult'] != 'S']
data.shape

(14326, 242)

In [89]:
data['passResult'] = data['passResult'].apply(lambda x: 1 if x == 'C' else 0)

In [90]:
y = data['passResult']
data1 = data.copy()
data1 = data1.drop(['gameId', 'playId', 'passResult', 'undefined'], axis=1)

In [91]:
data1.head()

,yardsToGo,score_diff_home_vs_away,a_1_def_from_qb,a_2_def_from_qb,a_3_def_from_qb,a_4_def_from_qb,a_5_def_from_qb,dir_1_def_from_qb,dir_2_def_from_qb,dir_3_def_from_qb,dir_4_def_from_qb,dir_5_def_from_qb,dis_1_def_from_qb,dis_2_def_from_qb,dis_3_def_from_qb,dis_4_def_from_qb,dis_5_def_from_qb,distance_from_qb_1_def_from_qb,distance_from_qb_2_def_from_qb,distance_from_qb_3_def_from_qb,distance_from_qb_4_def_from_qb,distance_from_qb_5_def_from_qb,height_1_def_from_qb,height_2_def_from_qb,height_3_def_from_qb,height_4_def_from_qb,height_5_def_from_qb,o_1_def_from_qb,o_2_def_from_qb,o_3_def_from_qb,o_4_def_from_qb,o_5_def_from_qb,s_1_def_from_qb,s_2_def_from_qb,s_3_def_from_qb,s_4_def_from_qb,s_5_def_from_qb,weight_1_def_from_qb,weight_2_def_from_qb,weight_3_def_from_qb,weight_4_def_from_qb,weight_5_def_from_qb,x_1_def_from_qb,x_2_def_from_qb,x_3_def_from_qb,x_4_def_from_qb,x_5_def_from_qb,y_1_def_from_qb,y_2_def_from_qb,y_3_def_from_qb,y_4_def_from_qb,y_5_def_from_qb,a_1_def_from_target_receiver,a_2_def_from_target_receiver,a_3_def_from_target_receiver,defender_distance_from_target_receiver_1_def_from_target_receiver,defender_distance_from_target_receiver_2_def_from_target_receiver,defender_distance_from_target_receiver_3_def_from_target_receiver,dir_1_def_from_target_receiver,dir_2_def_from_target_receiver,dir_3_def_from_target_receiver,dis_1_def_from_target_receiver,dis_2_def_from_target_receiver,dis_3_def_from_target_receiver,height_1_def_from_target_receiver,height_2_def_from_target_receiver,height_3_def_from_target_receiver,o_1_def_from_target_receiver,o_2_def_from_target_receiver,o_3_def_from_target_receiver,s_1_def_from_target_receiver,s_2_def_from_target_receiver,s_3_def_from_target_receiver,weight_1_def_from_target_receiver,weight_2_def_from_target_receiver,weight_3_def_from_target_receiver,x_1_def_from_target_receiver,x_2_def_from_target_receiver,x_3_def_from_target_receiver,y_1_def_from_target_receiver,y_2_def_from_target_receiver,y_3_def_from_target_receiver,time_to_throw_sec,x_x,y_x,x_y,y_y,throw_distance,target_receiver_nflId,targeted_receiver_pass_forward_x,targeted_receiver_pass_forward_y,targeted_receiver_pass_forward_s,targeted_receiver_pass_forward_a,targeted_receiver_pass_forward_dis,targeted_receiver_pass_forward_o,targeted_receiver_pass_forward_dir,targeted_receiver_distance_to_ball,EMPTY,I_FORM,JUMBO,PISTOL,SHOTGUN,SINGLEBACK,WILDCAT,"0 RB, 0 TE, 5 WR","0 RB, 1 TE, 4 WR","0 RB, 2 TE, 3 WR","0 RB, 3 TE, 2 WR","1 RB, 0 TE, 3 WR,1 DB","1 RB, 0 TE, 3 WR,1 DL","1 RB, 0 TE, 4 WR","1 RB, 1 TE, 2 WR,1 DL","1 RB, 1 TE, 3 WR","1 RB, 2 TE, 1 WR,1 DB","1 RB, 2 TE, 1 WR,1 DL","1 RB, 2 TE, 2 WR","1 RB, 3 TE, 1 WR","1 RB, 4 TE, 0 WR","2 QB, 0 RB, 1 TE, 3 WR","2 QB, 1 RB, 0 TE, 3 WR","2 QB, 1 RB, 1 TE, 1 WR,1 DL","2 QB, 1 RB, 1 TE, 2 WR","2 QB, 1 RB, 2 TE, 1 WR","2 QB, 2 RB, 1 TE, 1 WR","2 QB, 2 RB, 2 TE, 0 WR","2 RB, 0 TE, 3 WR","2 RB, 1 TE, 2 WR","2 RB, 2 TE, 1 WR","2 RB, 3 TE, 0 WR","3 RB, 0 TE, 2 WR","3 RB, 1 TE, 1 WR","3 RB, 2 TE, 0 WR","4 RB, 1 TE, 0 WR","6 OL, 0 RB, 0 TE, 0 WR,4 DL","6 OL, 1 RB, 0 TE, 3 WR","6 OL, 1 RB, 1 TE, 2 WR","6 OL, 1 RB, 2 TE, 0 WR,1 DL","6 OL, 1 RB, 2 TE, 1 WR","6 OL, 1 RB, 3 TE, 0 WR","6 OL, 2 RB, 0 TE, 2 WR","6 OL, 2 RB, 1 TE, 0 WR,1 DL","6 OL, 2 RB, 1 TE, 1 WR","6 OL, 2 RB, 2 TE, 0 WR","0 DL, 4 LB, 7 DB","0 DL, 5 LB, 6 DB","0 DL, 6 LB, 5 DB","1 DL, 2 LB, 7 DB","1 DL, 3 LB, 6 DB, 1 WR","1 DL, 3 LB, 7 DB","1 DL, 4 LB, 5 DB","1 DL, 4 LB, 6 DB","1 DL, 5 LB, 5 DB","2 DL, 2 LB, 7 DB","2 DL, 3 LB, 5 DB","2 DL, 3 LB, 5 DB, 1 WR","2 DL, 3 LB, 6 DB","2 DL, 4 LB, 4 DB","2 DL, 4 LB, 5 DB","3 DL, 0 LB, 8 DB","3 DL, 1 LB, 6 DB, 1 TE","3 DL, 1 LB, 6 DB, 1 WR","3 DL, 1 LB, 7 DB","3 DL, 2 LB, 5 DB","3 DL, 2 LB, 5 DB, 1 WR","3 DL, 2 LB, 6 DB","3 DL, 3 LB, 5 DB","3 DL, 4 LB, 4 DB","3 DL, 5 LB, 3 DB","4 DL, 0 LB, 7 DB","4 DL, 1 LB, 6 DB","4 DL, 2 LB, 4 DB","4 DL, 2 LB, 5 DB","4 DL, 3 LB, 4 DB","4 DL, 4 LB, 3 DB","4 DL, 5 LB, 2 DB","5 DL, 1 LB, 5 DB","5 DL, 2 LB, 4 DB","5 DL, 3 LB, 2 DB, 1 OL","5 DL, 3 LB, 3

In [92]:
data1.shape

(14326, 238)

In [93]:
X = data1

### split data (80/20)

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Basic Decision Tree Model

In [95]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

In [97]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (testset) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print()
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))

accuracy Score (testset) for Decision Tree:0.778437

Confusion Matrix for Decision Tree
[[ 305  314]
 [ 321 1926]]


### XGBoost Model with GridSearchCV for hyperparameter tuning

In [ ]:

model = xgb.XGBClassifier(eval_metric='logloss')

param_grid = {
    'subsample': [0.5, 0.7, 1],
    'max_depth': [3,5,7,10],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [50,100,200]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='accuracy', cv=5, verbose=1, n_jobs=1)

grid_search.fit(X_train, y_train)

y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_}")


In [98]:
### XGBoost Model with BayesSearchCV for hyperparameter tuning
## (supposedly faster)

In [ ]:
from skopt import BayesSearchCV
from sklearn.model_selection import KFold

In [ ]:
cv = KFold(n_splits=5)
xgb_2 = xgb.XGBClassifier(eval_metric='logloss')
xgb_param_grid = {
    'subsample': (0.5, 1.0, 'uniform'),
    'max_depth': (3,15),
    'learning_rate': (0.001, 0.5, 'uniform'),
    'n_estimators': (50, 1000)
}
xgb_search2 = BayesSearchCV(
    estimator=xgb_2,
    search_spaces=xgb_param_grid,
    n_iter=50,  # Number of iterations (how many trials to run)
    cv=cv,  # 3-fold cross-validation
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)
xgb_search2.fit(X_train, y_train)

In [ ]:
# Get the best hyperparameters and the best score from cross-validation
print("Best Hyperparameters:", xgb_2.best_params_)
print("Best Cross-Validation Score:", xgb_2.best_score_)

# Evaluate on the test set
y_pred = xgb_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

In [ ]:
best_model = grid_search.best_estimator_
importances = best_model.feature_importances_
feature_names = data.columns

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

print(feature_importance_df.head(20))


### Random Forest Model with GridSearchCV for hyperparameter tuning

In [ ]:
rf_param_grid = {
    'max_depth': [10, 20, 50],
    'max_features': ['log2', 'sqrt'],
    'n_estimators': [500, 750, 1000],
}
rf_model = RandomForestClassifier()
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, 
                           scoring='accuracy', cv=5, verbose=1, n_jobs=1)

rf_grid_search.fit(X_train, y_train)

rf_y_pred = rf_grid_search.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_y_pred)

print(f"Accuracy: {rf_accuracy}")
print(f"Best Parameters: {rf_grid_search.best_params_}")
print(f"Best Cross-Validation Score: {rf_grid_search.best_score_}")

# Model depth of target (IGNORE)

In [ ]:
data2 = data[data['passResult'] == 1]
data2.head(10)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
data2 = data2.merge(plays[['gameId','playId','offensePlayResult']],
                 on=['gameId','playId'], how='inner')
data2.head(10)

In [ ]:
y2 = data2['offensePlayResult']
X2 = data2.copy()
X2 = X2.drop(['gameId', 'playId', 'passResult', 'offensePlayResult'], axis=1)

In [ ]:
X2.head(10)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
y2.head

In [ ]:
cv = KFold(n_splits=5)
log_param_grid = {
    'C': [0.001],
    'penalty': ['l2'],
    'solver': ['newton-cg', 'liblinear']
}
log_grid_search = GridSearchCV(LogisticRegression(max_iter=1000), log_param_grid, 
                               scoring='neg_root_mean_squared_error', 
                               cv=cv, n_jobs=1, verbose=1)
        
log_grid_search.fit(X2, y2)

log_rmse = math.sqrt(log_grid_search.best_score_)

print(f"Accuracy: {log_rmse}")
print(f"Best Parameters: {log_grid_search.best_params_}")
print(f"Best Cross-Validation Score: {log_grid_search.best_score_}")

In [ ]:
log_param_grid = {
    'C': [0.001, 0.05, 0.1, 1],
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']
}

In [ ]:
log_param_grid = {
    'C': [0.001],
    'penalty': ['l1'],
    'solver': ['liblinear']
}

log_bayes_search = BayesSearchCV(
    estimator=LogisticRegression(),
    search_spaces=log_param_grid,
    n_iter=10,  # Number of iterations (how many trials to run)
    cv=cv,  # 3-fold cross-validation
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)
log_bayes_search.fit(X2_train, y2_train)

In [ ]:
# Get the best parameters and the best score
print("Best Parameters:", bayes_search.best_params_)
print("Best Cross-Validation Score:", bayes_search.best_score_)

# Evaluate on the test set
y_pred = bayes_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test Mean Squared Error:", mse)

In [ ]:
xgb_reg = xgb.XGBRegressor()
xgb_param_grid = {
    'subsample': (0.5, 1.0, 'uniform'),
    'max_depth': (3,15),
    'learning_rate': (0.001, 0.5, 'uniform'),
    'n_estimators': (50, 1000)
}
xgb_search = BayesSearchCV(
    estimator=xgb_reg,
    search_spaces=xgb_param_grid,
    n_iter=50,  # Number of iterations (how many trials to run)
    cv=cv,  # 3-fold cross-validation
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)
xgb_search.fit(X2_train, y2_train)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# Get the best hyperparameters and the best score from cross-validation
print("Best Hyperparameters:", xgb_search.best_params_)
print("Best Cross-Validation Score:", xgb_search.best_score_)

# Evaluate on the test set
y2_pred = xgb_search.predict(X2_test)
mse = mean_squared_error(y2_test, y2_pred)
print("Test Mean Squared Error:", mse)